In [ ]:
import optuna
from sindy import *
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt

## Simulate Lorenz Attractor

In [ ]:
# system parametera
sigma = 10.0
rho = 28.0
beta = 8/3
params = np.array([sigma, rho, beta])

# state => [x, y, z]
X0 = np.array([0, 1, 20])

# timesteps
n_steps = 10000
t = np.linspace(0, 50, n_steps)

# simulated states and timsteps data
X, t = simulate_lorenz_attractor(
    initial_state=X0, 
    timesteps=t, 
    params=params
)

X.shape, t.shape

## Compute Derivatives Numerically

In [ ]:
# numerically computed derivatives of system
numerical_derivatives = compute_derivatives(
    data=X, 
    timesteps=t,
    h=1
)

# actual derivatives of system
actual_derivatives = np.array([
    lorenz_ode(t, x, sigma, rho, beta) for t, x in zip(t, X)
])

## Simulaate and Add Periodic Control Input to the Lorenz System state space

In [ ]:
# periodic control input
U = 10 * np.sin(t / 5)
plt.figure(figsize=(10, 3))
plt.plot(t, U)
plt.title(r'Input $U(t)$')
plt.xlabel(r'$t$')
plt.grid()

In [ ]:
# Add periodic control input to state matrix
X_U = np.concatenate([X, U.reshape(-1, 1)], axis=1)
X_U.shape

## Compute Polynomial Features

In [ ]:
# polynomial features
X_poly = compute_poly_features(
    data=X_U,
    degree=2, 
    column_names=["X", "Y", "Z", "U"]
)

X_poly.head()

## Define Data Scaler Object For scaling Data to a specific Range

In [ ]:
# data scaler object
scaler = StandardScaler()

## Tune the relevant Hyper-parameters with Optuna

In [ ]:
# hyper-parameter tuning space
alpha_space = (0.01, 0.5)
max_iter_space = (1000, 2000)
max_features_space = (3, 7)
n_cv = 10
threshold=-np.inf

# construct objective to find the best value of alpha max_iter and max_features
# that best minimize the 2 norm of the difference between the estimated 
# derivatives and the numerically computed derivatives.
objective = lambda trial: optimize_objective(
    trial=trial, 
    poly_features=X_poly.values, 
    derivatives=numerical_derivatives, 
    alpha_space=alpha_space, 
    max_iter_space=max_iter_space,
    max_features_space=max_features_space,
    n_cv=n_cv,
    threshold=threshold,
    scaler=scaler,
)

# create optuna study
study = optuna.create_study(
    study_name="SINDy",
    direction="minimize",
    pruner=optuna.pruners.MedianPruner(),
    sampler=optuna.samplers.TPESampler(),
)

# optimize study objective
study.optimize(objective, n_trials=20)

## Compute Sparse Coefficients and Get Selected Features

In [ ]:
# compute sparse coeffs and get features index
sindy_coeffs, selected_features = compute_linear_operator(
    poly_features=X_poly.values,
    derivatives=numerical_derivatives,
    alpha=study.best_trial.params["alpha"],
    max_iter=study.best_trial.params["max_iter"],
    max_features=study.best_trial.params["max_features"],
    n_cv=n_cv,
    threshold=threshold,
    scaler=scaler
)

BETA = sindy_coeffs.T.round(3)

In [ ]:
# get relevant features
relevant_cols = X_poly.columns[selected_features]
X_prime = X_poly[relevant_cols]

# normalise data for visual validation
X_prime = scaler.fit_transform(X_prime)
actual_derivatives = scaler.fit_transform(actual_derivatives)
numerical_derivatives = scaler.fit_transform(numerical_derivatives)

print(
    f"alpha: {study.best_trial.params['alpha']} \n",
    f"max_iter: {study.best_trial.params['max_iter']} \n",
    f"max_features: {study.best_trial.params['max_features']}",
    f"Relevant Columns: \n{relevant_cols} \n\n", 
    f"Sparse Linear Operator: \n{BETA}",
)

## Use Computed Linear Operator (Sparse Coefficient) to Estimate derivatives

In [ ]:
# estimate derivatives with linear combination
# of relevant features and sparse linear operator (beta)
estimated_derivatives = X_prime @ BETA

## Visualise and Compare Derivatives

In [ ]:
# plot data
fig, axs = plt.subplots(1, 4, figsize=(20, 7), subplot_kw=dict(projection='3d'))
axs[0].plot(X[:, 0], X[:, 1], X[:, 2])
axs[0].set_title(f"Lorenz Attractor")

axs[1].plot(actual_derivatives[:, 0], actual_derivatives[:, 1], actual_derivatives[:, 2])
axs[1].set_title("Actual Derivatives")

axs[2].plot(numerical_derivatives[:, 0], numerical_derivatives[:, 1], numerical_derivatives[:, 2])
axs[2].set_title("Numerically Computed Derivatives")

axs[3].plot(estimated_derivatives[:, 0], estimated_derivatives[:, 1], estimated_derivatives[:, 2])
axs[3].set_title("Predicted Derivatives")
plt.show()